# Formel 1 Datenanalyse mit OpenF1 API

In diesem Notebook werden wir die OpenF1 API nutzen, um Formel 1 Daten zu analysieren. Die API bietet Zugriff auf Echtzeit-Renndaten, Rundenzeiten, und mehr.

## Was wir lernen werden:
- API Abfragen mit requests
- Datenverarbeitung mit pandas
- Visualisierung mit plotly
- Praktische Datenanalyse an echten F1-Daten

In [1]:
# Benötigte Bibliotheken importieren
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import numpy as np

## Daten von der OpenF1 API abrufen

Wir werden zunächst Daten vom letzten Rennen abrufen. Die API bietet verschiedene Endpunkte für:
- Rundenzeiten
- Fahrerpositionen
- Boxenstopps
- und mehr

## Analyse der Rundenzeiten

Schauen wir uns die Entwicklung der Rundenzeiten über das Rennen hinweg an.

In [8]:
API_BASE_URL = "https://api.openf1.org/v1/"

In [9]:
import io

# Fetch sessions for 2023 in CSV format and load into a pandas DataFrame

params = {'year': 2023, 'csv': 'true'}
url = f"{API_BASE_URL}sessions"  # API_BASE_URL is defined in the notebook

resp = requests.get(url, params=params, timeout=15)
resp.raise_for_status()  # raise an error if the request failed

df_sessions_2023 = pd.read_csv(io.StringIO(resp.text))
df_sessions_2023.head()

,circuit_key,circuit_short_name,country_code,country_key,country_name,date_end,date_start,gmt_offset,location,meeting_key,session_key,session_name,session_type,year
0,63,Sakhir,BRN,36,Bahrain,2023-02-23 16:30:00+00:00,2023-02-23 07:00:00+00:00,03:00:00,Sakhir,1140,9222,Practice 1,Practice,2023
1,63,Sakhir,BRN,36,Bahrain,2023-02-24 16:30:00+00:00,2023-02-24 07:00:00+00:00,03:00:00,Sakhir,1140,7763,Practice 2,Practice,2023
2,63,Sakhir,BRN,36,Bahrain,2023-02-25 16:30:00+00:00,2023-02-25 07:00:00+00:00,03:00:00,Sakhir,1140,7764,Practice 3,Practice,2023
3,63,Sakhir,BRN,36,Bahrain,2023-03-03 12:30:00+00:00,2023-03-03 11:30:00+00:00,03:00:00,Sakhir,1141,7765,Practice 1,Practice,2023
4,63,Sakhir,BRN,36,Bahrain,2023-03-03 16:00:00+00:00,2023-03-03 15:00:00+00:00,03:00:00,Sakhir,1141,7766,Practice 2,Practice,2023


In [10]:
# find the latest session in df_sessions_2023 by start time
latest_idx = pd.to_datetime(df_sessions_2023['date_start']).idxmax()
latest_session = df_sessions_2023.loc[latest_idx]
latest_session

circuit_key                                  70
circuit_short_name           Yas Marina Circuit
country_code                                UAE
country_key                                  21
country_name               United Arab Emirates
date_end              2023-11-26 15:00:00+00:00
date_start            2023-11-26 13:00:00+00:00
gmt_offset                             04:00:00
location                             Yas Island
meeting_key                                1226
session_key                                9197
session_name                               Race
session_type                               Race
year                                       2023
Name: 112, dtype: object

In [22]:
from io import StringIO

session_key = 9197  # bekannter, funktionierender Testwert

url = f"{API_BASE_URL}drivers?session_key={session_key}"
print(f"Abfrage: {url}")
resp = requests.get(url, timeout=15)
try:
    resp.raise_for_status()
except Exception as e:
    print(f"HTTP-Fehler: {e}")
    drivers_in_session = pd.DataFrame()
else:
    text = resp.text.strip()
    # JSON-Fall
    if text.startswith('{') or text.startswith('[') or 'application/json' in resp.headers.get('content-type', ''):
        data = resp.json()
        # Prüfe bekannte Wrapper-Felder
        drivers_in_session = None
        if isinstance(data, dict):
            for key in ('data', 'drivers', 'items', 'results'):
                if key in data and isinstance(data[key], list):
                    drivers_in_session = pd.DataFrame(data[key])
                    break
            if drivers_in_session is None:
                # dict als einzelne Zeile versuchen
                try:
                    drivers_in_session = pd.DataFrame([data])
                except Exception:
                    drivers_in_session = pd.DataFrame()
        elif isinstance(data, list):
            drivers_in_session = pd.DataFrame(data)
        else:
            drivers_in_session = pd.DataFrame()
    else:
        # CSV/Text-Fallback
        try:
            drivers_in_session = pd.read_csv(StringIO(resp.text))
        except Exception:
            drivers_in_session = pd.DataFrame()

if drivers_in_session is None or drivers_in_session.empty:
    print(f'Keine Fahrerinformationen gefunden für session_key={session_key}')
else:
    # Kurz zeigen; weitere Bereinigung/Normalisierung optional
    display(drivers_in_session.head(200))

Abfrage: https://api.openf1.org/v1/drivers?session_key=9197


,meeting_key,session_key,driver_number,broadcast_name,full_name,name_acronym,team_name,team_colour,first_name,last_name,headshot_url,country_code
0,1226,9197,1,M VERSTAPPEN,Max VERSTAPPEN,VER,Red Bull Racing,3671C6,Max,Verstappen,https://www.formula1.com/content/dam/fom-websi...,NED
1,1226,9197,2,L SARGEANT,Logan SARGEANT,SAR,Williams,37BEDD,Logan,Sargeant,https://www.formula1.com/content/dam/fom-websi...,USA
2,1226,9197,3,D RICCIARDO,Daniel RICCIARDO,RIC,AlphaTauri,5E8FAA,Daniel,Ricciardo,None,AUS
3,1226,9197,4,L NORRIS,Lando NORRIS,NOR,McLaren,F58020,Lando,Norris,https://www.formula1.com/content/dam/fom-websi...,GBR
4,1226,9197,10,P GASLY,Pierre GASLY,GAS,Alpine,2293D1,Pierre,Gasly,https://www.formula1.com/content/dam/fom-websi...,FRA
5,1226,9197,11,S PEREZ,Sergio PEREZ,PER,Red Bull Racing,3671C6,Sergio,Perez,https://www.formula1.com/content/dam/fom-websi...,MEX
6,1226,9197,14,F ALONSO,Fernando ALONSO,ALO,Aston Martin,358C75,Fernando,Alonso,https://www.formula1.com/content/dam/fom-websi...,ESP
7,1226,9197,16,C LECLERC,Charles LECLERC,LEC,Ferrari,F91536,Charles,Leclerc,https://www.formula1.com/content/dam/fom-websi...,MON
8,1226,9197,18,L STROLL,Lance STROLL,STR,Aston Martin,358C75,Lance,Stroll,https://www.formula1.com/content/dam/fom-websi...,CAN
9,1226,9197,20,K MAGNUSSEN,Kevin MAGNUSSEN,MAG,Haas F1 Team,B6BABD,Kevin,Magnussen,https://www.formula1.com/content/dam/fom-websi...,DEN
